In [ ]:
dat[dat['response'].duplicated()] # The one duplicated response is the duplicated one from before


re.sub(r"\/*u\/[\S]+", 'they', TEXT) # This works to replace usernames with they

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer

